In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Método para estimación de datos faltantes

In [ ]:
data = pd.read_csv("./dataset.csv", header=0)


MISSING_VALUE = 999.99


def estimate_missing_values(data, k):

    # Separamos los datos en dos conjuntos:
    # - Completos: los datos que no tienen valores en MISSING_VALUE
    complete_df = data
    complete_df = complete_df[~complete_df.isin([MISSING_VALUE]).any(axis=1)]

    # - Incompletos: los datos que tienen al menos 1 valor en MISSING_VALUE
    incomplete_df = data
    incomplete_df = incomplete_df[incomplete_df.isin(
        [MISSING_VALUE]).any(axis=1)]

    # Estandarizamos ambos conjuntos
    scaler = StandardScaler()
    std_complete_df = scaler.fit_transform(complete_df)
    std_incomplete_df = scaler.fit_transform(incomplete_df)

    estimated_data = data.copy()

    # Calculamos la distancia euclidea entre cada dato incompleto y todos los datos completos
    for incomplete_row in std_incomplete_df:
        distances = []
        missing_column_names = incomplete_row.columns[incomplete_row.isin(
            [MISSING_VALUE]).any(axis=1)]

        # Eliminamos los datos faltantes
        incomplete_row = incomplete_row.drop(missing_column_names, axis=1)

        for i, complete_row in enumerate(std_complete_df):
            # Eliminamos los datos faltantes
            complete_row = complete_row.drop(missing_column_names, axis=1)

            # Almacenamos distancia para ordenamiento e índice de fila 
            distances[i] = (np.linalg.norm(incomplete_row - complete_row), i)

        # Calculamos los k vecinos más cercanos
        k_nearest_neighbors_indexes = map(lambda x: x[0], sorted(
            distances, key=lambda distance: distance[0])[:k])

        k_nearest_neighbors_df = complete_df.filter(
            items=k_nearest_neighbors_indexes, axis=0)

        # Calculamos la media de los k vecinos más cercanos para cada dato incompleto
        for missing_column_name in missing_column_names:
            estimated_data.loc[incomplete_row.name, missing_column_name] = k_nearest_neighbors_df[missing_column_name].mean()
        
    return estimated_data

# Eliminar columna "sexo"
data = data.drop(["Sexo"], axis=1)

estimated = estimate_missing_values(data, k=5)
display(estimated)
# print(distances)
# print(min(distances))
# print(complete_df.iloc[distances.index(min(distances))])
# print("\n")


### Boxplots de cada variable

In [ ]:

# Plot de data sin escalar
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))
axes[0].boxplot(complete_df["Calorías"])
axes[0].set_title("Calorías")

axes[1].boxplot(complete_df["Alcohol"])
axes[1].set_title("Alcohol")

axes[2].boxplot(complete_df["Grasas_sat"])
axes[2].set_title("Grasas_sat")

### Comparación de datos sin estandarizar vs estandarizados mediante boxplots

In [ ]:
# Plot de data sin escalar
data_without_genre = complete_df.drop("Sexo", axis=1)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].boxplot(data_without_genre)
axes[0].set_xticks([1, 2, 3], data_without_genre.columns)
axes[0].set_title("Boxplot de datos sin estandarizar")


scaler = StandardScaler().fit(data_without_genre)
scaled_data = scaler.transform(data_without_genre)
# Plot de data escalada
axes[1].boxplot(pd.DataFrame(scaled_data))
axes[1].set_xticks([1, 2, 3], data_without_genre.columns)
axes[1].set_title("Boxplot de datos estandarizados")

### Reemplazo de datos faltantes